In [ ]:
import datetime
from pprint import pprint

In [ ]:
import ee
ee.Initialize()
import rendvi
from rendvi import eeCollections

The exporting of the reNDVI data is very memory intensive because of the time series processing. Therefore, the exports should be done in batches of smaller time ranges. Meaning that the user should manually change the start and end years for the processing. The only thing to be aware of is that two dekads at the beginning and end of the time periods are needed to perform the despiking/smoothing.

Note here we have a longer time period, this is because we will use historical data to forecast out what we expect from historically observed. Typically a ten year period is good enough to calculate the trends but not long enough to givememory errors. This will be trimmed down to a more relevant time period for despiking/smoothing and export.

In [ ]:
# time information to handle image collection
iniYear = 2015
endYear = 2020

today = ee.Date(datetime.datetime.now().strftime('%Y-%m-%d')).advance(-7,'day')

# convert start and end dates to EE date objects
eeIni = ee.Date.fromYMD(iniYear,1,1,)
eeEnd = ee.Date.fromYMD(endYear,12,31)

# make list of years for loop processing
years = ee.List.sequence(iniYear,endYear)

reNDVI supports the processing of both MODIS and VIIRS. The processing is more or less the same, the only difference is how the masking is applied. The follow code blocks provide the preprocessing for MODIS and VIIRS, so comment the block of the product to omit.

In [ ]:
mod = eeCollections.MOD09GQ['imageCollection']
mod1km = eeCollections.MOD09GA['imageCollection']

masked = rendvi.Masking.applyModis(mod,mod1km)
withNdvi = rendvi.Utils.addNDBand(
    masked,
    b1=eeCollections.MOD09GQ['nir'],
    b2=eeCollections.MOD09GQ['red'],
    outName='ndvi'
).filterDate(eeIni,eeEnd)

In [ ]:
# preprocessing for VIIRS
viirs = eeCollections.VNP09GA['imageCollection']

masked = rendvi.Masking.applyViirs(viirs)
withNdvi = rendvi.Utils.addNDBand(
    masked,
    b1=eeCollections.VNP09GA['nir'],
    b2=eeCollections.VNP09GA['red'],
    outName='ndvi'
).filterDate(eeIni,eeEnd)

In [ ]:
# instantiate a rendvi data processing object
# this has special methods for processing the data
full = rendvi.Rendvi(withNdvi,'ndvi')

In [ ]:
# calculate the dekad imagery from the full collection
dekads = full.getDekadImages(includeQa=True)

In [ ]:
# load in the climatology data for the back fill process
climo = ee.ImageCollection("projects/servir-e-sa/rangelands/reNDVI_climatology_africa")

In [ ]:
# apply despiking process
despiked = dekads.applyDespike(window=30,step=10)

For the real-time processing of the data we will use a harmonic model to provide the data for the despiking and Before applying the despike and smoothing algorithms, 

In [ ]:
# import the forecasting module
from rendvi import forecast

In [ ]:
# instantiate an object to fit and predict the forecast
# uses 3 harmonic cycles here. Kenya has two rainy seasons
# add additional cycle for model fitting
fm = forecast.Harmonics(nCycles=3)

In [ ]:
# fit the forecast on the despiked data
fm.fit(despiked)

In [ ]:
# set the start time and end time to forecast ndvi
# based on the harmonic model
# typically a range of 60 days (6 dekads) will be good
forecastStart = ""
forecastEnd = ""

In [ ]:
# get an empty collection of future times for prediction
futureColl = fm.getDummyCollection(forecastStart, forecastEnd)

In [ ]:
# apply prediction on in the future
prediction = fm.predict(futureColl)

In [ ]:
# get the last 60 days of the observed 
nrtcoll = withNdvi.limit(60, "system:time_start", False).merge(prediction)

In [ ]:
# instantiate a new rendvi data processing object
# this has the forecast data appended
full_forecast = rendvi.Rendvi(nrtcoll,'ndvi')

In [ ]:
# apply backfill algorithm using climatology
backFilled = despiked.climatologyBackFill(full_forecast,keepBandPattern="^(de|pct|nClear).*")

In [ ]:
# apply spatial smoothing process (optional)
# this looks are areas of discontinuity, pixels > 1.5 std from moving window mean
# and smooths. This makes the back filled data more spatially consistent with others
kernel = ee.Kernel.square(7.5,"pixels")
spatialSmoothed = backFilled.spatialSmoothing(kernel,zThreshold=1,keepBandPattern="^(clima|de|pct|nClear|t).*")

In [ ]:
# apply moving window smoothing algorithm in time
smoothed = spatialSmoothed.applySmoothing(window=50,keepBandPattern="^(clima|de|pct|nClear|sp).*")

In [ ]:
# function to post-process the data into storage efficient formats
# and mask out ocean areas
def formatOutput(img):
    ndvi = img.select('ndvi').multiply(10000).int16()
    others = img.select('^(clima|de|pct|sp|tem).*').multiply(100).uint8()
    
    return ee.Image.cat([ndvi,others]).updateMask(landMask)\
        .set('system:time_start',img.date().millis())



In [ ]:
# read in a land/water mask and apply post processing function
landMask = ee.Image("users/kelmarkert/public/landMask").select("land")
outputs = smoothed.imageCollection.filterDate("2007-02-01","2007-03-21").map(formatOutput).sort('system:time_start')

In [ ]:
# change the path to theimage collection for the climatology imagery
exportAsset = "projects/servir-e-sa/rangelands/reNDVI"

In [ ]:
# change the bounding box coordinates to the region for export
# coordinates should be W,S,E,N
# the following is the bounding box for Kenya
bbox = [33,-5,42,6]

In [ ]:
# change the data contact to the individual responsible for 
# managing the data
data_contact = "username@nasa.gov"

In [ ]:
# change the name of the data name prefix (optional)
data_prefix = "MOD_reNDVI"

In [ ]:
# change to the version/release number of interest (optional)
version_number = 0

In [ ]:
# create metadata and pyramiding policies for the data
metadataDict = dict(contact=data_contact,scale=0.0001,offset=0,version=version_number)
pyramidingDict = dict(ndvi_mean="mean",ndvi_std="mean",count="mode")
exportRegion = ee.Geometry.Rectangle(bbox,'epsg:4326',False)

In [ ]:
# create metadata and pyramiding policies for the data
exportRegion = ee.Geometry.Rectangle(bbox,'epsg:4326',False)
metadataDict = dict(contact=data_contact,ndviScale=0.0001,otherScale=0.01,offset=0,version=version_number,creationDate=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%s"),"status":"provisional")
pyramidingDict = {'.default':"mean",'despiked':'mode','climatologyFilled':'mode','temporalFilled':'mode','spatialSmoothed':'mode'}

In [ ]:
rendvi.batchExport(outputs, 
                   exportRegion, 
                   exportAsset, 
                   prefix="MOD_reNDVI", 
                   suffix="v0", 
                   scale=250, 
                   crs='EPSG:4326',
                   metadata=metadataDict, 
                   pyramiding=pyramidingDict
                  )